In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as Drawgeo
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def generate_mesh(h, inner_rect_size, PML_size):
    
    inner_rect=WorkPlane().RectangleC(inner_rect_size,inner_rect_size).Face()
    scatterer = WorkPlane().Circle(0,0,1).Face()

    inner_rect.edges.name = 'innerbnd'
    scatterer.edges.name = 'scabnd'

    inner = inner_rect - scatterer

    #Drawgeo(inner)

    wp2=WorkPlane().RectangleC(inner_rect_size+inner_rect_size,inner_rect_size+inner_rect_size).RectangleC(5,5).Reverse()
    outer = wp2.Face()

    outer.edges.name = 'outerbnd'
    #inner.edges.name = 'innerbnd'
    inner.faces.name ='inner'
    outer.faces.name = 'pmlregion'

    #Drawgeo(Glue([outer,inner]))

    geo = OCCGeometry(Glue([inner, outer]), dim=2)
    mesh = Mesh(geo.GenerateMesh (maxh=h))
    mesh.Curve(5)
    return mesh

inner_rect_size = 5
PML_size = 5
h = 0.3
mesh = generate_mesh(h, inner_rect_size, PML_size)
Draw(mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [15]:
K = CF((3.14159 * 2, 0))
phasor = exp(1j * ((K[1] * x) + (K[0] * z)))
ez = 1 * phasor
Draw(ez, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [16]:
fes = HCurl(mesh, order=4, complex=True, dirichlet='scabnd')
u = fes.TrialFunction()
v = fes.TestFunction()

omega = sqrt(K[0]**2 + K[1]**2)

a = BilinearForm(fes, symmetric=True)
a += curl(u)*curl(v)*dx - omega**2*u*v*dx
#a += -1j*omega*u*v*ds("outerbnd")
a.Assemble()


b = LinearForm(fes)
b += SymbolicLFI(0)
b.Assemble()

scat = GridFunction(fes)
scat.Set(-ez, BND)

r = b.vec - a.mat * scat.vec
scat.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r

Draw(scat, mesh)
#Draw(ez, mesh)
#Draw(scat - ez, mesh)

NgException: Error in SetValues: gridfunction-dim = 2, but coefficient-dim = 1

In [9]:
ez

In [10]:
ez.dim

1

In [12]:
ez[1]

IndexError: 